In [1]:
from translator import Translator
import torch

In [2]:
import word2vec
out_vac_size = 20000
device = torch.device('cuda')
print("loading embedding")
embedding_loader = word2vec.WordEmbeddingLoader(device, "../embeddings/sgns.merge/sgns.merge.word", out_vocab_size=out_vac_size)

loading embedding


/home/jovyan/workspace/lstm-translator/word2vec.py:44: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  self.zh_vectors.append(torch.from_numpy(self.model.get_vector(k))[:embeddings].unsqueeze(0).to(device))


In [6]:
embeddings = 300
hiddens = 600
n_layers = 4

# print("loading embedding")
# embedding_loader = word2vec.WordEmbeddingLoader(device, "../embeddings/sgns.merge/sgns.merge.word", out_vocab_size=out_vac_size)
# embedding_loader = word2vec.WordEmbeddingLoader("../embeddings/parellel_01.v2c")
# print("load embedding finished")

model_fname = "./models/seq2seq_1700436811.1431215_1400"
# model_fname = None
model = None
if not model_fname is None:
    print('loading model from ' + model_fname)
    model = torch.load(model_fname, map_location=device)
    print('model loaded')
else:
    model = Seq2Seq(device, embedding_loader, embeddings, hiddens, out_vac_size, n_layers, 0.5, 0.5).to(device)


loading model from ./models/seq2seq_1700436811.1431215_1400
model loaded


In [7]:
import tokenizer as tknzr
tokenizer = tknzr.Tokenizer()

x = []
x.append(tokenizer.tokenize("information sources and exchange on capacity building", lang= 'en'))
# x.append(tokenizer.tokenize("uk reports good data.", lang= 'en'))
print(x)
x_idx, x_len, _ = embedding_loader.scentences_to_indexes(x, lang="en")
print("x_idx:", x_idx)
print("x_len:", x_len)
x_emb = embedding_loader.get_embeddings(x_idx, lang="en")
# print("x_emb:", x_emb)
y_softmax = model.translate(x_emb, x_len, 5)
# print("y_softmax:", y_softmax)
y_idx = embedding_loader.softmax_to_indexes(y_softmax, lang="zh")
embedding_loader.index_to_scentence(y_idx, lang="zh")

[['information', 'sources', 'and', 'exchange', 'on', 'capacity', 'building']]
x_idx: tensor([[  1742,   9544,     19,   5523,    128,   5233,   4837, 163762]],
       device='cuda:0')
x_len: tensor([8], device='cuda:0')


[['<eos>', '<eos>', '<eos>', '<eos>', '<eos>']]